# Diffusion Training and Experimentation

This notebook demonstrates training Diffusion model on the MNIST dataset using a modular training pipeline implemented in PyTorch Lightning.


In [ ]:
!git clone https://github.com/Reennon/gen-ai-cv-lab-2-3.git
%cd gen-ai-cv-lab-2-3

In [ ]:
!pip install -r requirements.txt

In [ ]:
!git pull

In [ ]:
import os
import dotenv
import wandb
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
from src.training.diffusion_module import DiffusionLitModule
from src.training.trainer import train_model

from google.colab import userdata

In [ ]:
os.environ["WANDB_KEY"] = userdata.get("wandb_key")
!echo $WANDB_KEY >> .env

In [ ]:
dotenv.load_dotenv()

In [ ]:
parameters = OmegaConf.load("./params/diffusion.yml")
wandb.login(key=os.environ["WANDB_KEY"])

In [ ]:
wandb_project_name = "diffusion-mnist"
device = "gpu"

In [ ]:
# Prepare the MNIST dataset.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_train = MNIST(root='data', train=True, download=True, transform=transform)
mnist_val = MNIST(root='data', train=False, download=True, transform=transform)
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)


In [ ]:
hparams = parameters.hyperparameters

In [ ]:
model = train_model(DiffusionLitModule, hparams, train_loader, val_loader)

In [ ]:
# generate and visualize samples using DDIM.

samples = model.sample_ddim(num_steps=50, eta=0.0, shape=(16, 1, 28, 28))
import matplotlib.pyplot as plt
fig, axs = plt.subplots(4, 4, figsize=(6,6))
for i, ax in enumerate(axs.flatten()):
    img = (samples[i, 0].cpu().numpy() + 1) / 2.0  # Rescale from [-1,1] to [0,1]
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
wandb.finish()